# Setup

In [21]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


# Load data (must upload first if running on colab)
train_df = pd.read_csv('/content/train_sampled.csv')
dev_df = pd.read_csv('/content/dev_sampled.csv')
test_df = pd.read_csv('/content/test_sampled.csv')


# Check
train_df.head()

,label,text
0,0,"This KS isn't for a game, it's to sponsor his ..."
1,0,`NEWLINE_TOKENNEWLINE_TOKEN== Ownership of thi...
2,0,"""\nYesterday I made some quick changes to the ..."
3,1,#obama fucking massage sex vids https://t.co/d...
4,0,""" \n\n ==Disputed fair use rationale for Image..."


0 = text does NOT contain hateful speech

1 = text contains hateful speech

# Simple Baseline: Majority-class Classifier

### Model Setup & Training

In [7]:
# Define model

class MajorityClassModel:
    def __init__(self):

        self.majority_class = None


    def fit(self, y_train): # determine majority class from training labels
        self.majority_class = y_train.mode()[0]
        print(f"Majority class in training set = {self.majority_class}")


    def predict(self, X): # predict the majority class for any given samples
        return [self.majority_class] * len(X)





# Train
model = MajorityClassModel()
model.fit(train_df["label"])

Majority class in training set = 0


### Evaluation

In [8]:
# Predict on train and test values ---

y_pred_test = model.predict(test_df)
y_true_test = test_df['label'].values

y_pred_train = model.predict(train_df)
y_true_train = train_df["label"].values







# Compute & display metrics ---

def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    return acc, precision, recall, f1


test_acc, test_prec, test_rec, test_f1 = compute_metrics(y_true_test, y_pred_test)
train_acc, train_prec, train_rec, train_f1 = compute_metrics(y_true_train, y_pred_train)


metrics_df = pd.DataFrame({
    "Split": ["Train", "Test"],
    "Accuracy": [train_acc, test_acc],
    "Precision": [train_prec, test_prec],
    "Recall": [train_rec, test_rec],
    "F1": [train_f1, test_f1]
})

metrics_df.round(4)

,Split,Accuracy,Precision,Recall,F1
0,Train,0.7940,0.3970,0.5,0.4426
1,Test,0.7856,0.3928,0.5,0.4400


# Stronger Baseline: MetaHateBERT

### Model Setup

In [17]:
model = AutoModelForSequenceClassification.from_pretrained("irlab-udc/MetaHateBERT")
tokenizer = AutoTokenizer.from_pretrained("irlab-udc/MetaHateBERT")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()



def predict_texts(df): # predict function for easier inferencing
    preds = []
    with torch.no_grad():

        for text in df["text"].tolist():
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            preds.append(pred_label)

    return preds

### Evaluation

In [18]:
# Predict on train and test values ---

y_pred_train = predict_texts(train_df)
y_true_train = train_df["label"].values

y_pred_test = predict_texts(test_df)
y_true_test = test_df["label"].values






# Compute & display metrics ---

def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    return acc, precision, recall, f1

train_acc, train_prec, train_rec, train_f1 = compute_metrics(y_true_train, y_pred_train)
test_acc, test_prec, test_rec, test_f1 = compute_metrics(y_true_test, y_pred_test)


metrics_df = pd.DataFrame({
    "Split": ["Train", "Test"],
    "Accuracy": [train_acc, test_acc],
    "Precision": [train_prec, test_prec],
    "Recall": [train_rec, test_rec],
    "F1": [train_f1, test_f1]
})

metrics_df.round(4)

,Split,Accuracy,Precision,Recall,F1
0,Train,0.9392,0.9095,0.9031,0.9063
1,Test,0.9328,0.9097,0.8864,0.8973


### Alternate approach using transformers.pipeline API

You guys can decide if you prefer this approach or the earlier one

In [22]:
from transformers import pipeline

# Define model pipeline

classifier = pipeline(
    "text-classification",
    model="irlab-udc/MetaHateBERT",
    device=0,
    truncation=True,
    padding=True,
    max_length=512
)




# Predict on train and test values

label_map_pred = {"LABEL_0": 0, "LABEL_1": 1}

y_pred_train = [label_map_pred[classifier(text, truncation=True, max_length=512)[0]['label']] for text in train_df["text"].tolist()]
y_pred_test = [label_map_pred[classifier(text, truncation=True, max_length=512)[0]['label']] for text in test_df["text"].tolist()]

y_true_train = train_df["label"].values
y_true_test  = test_df["label"].values


train_acc, train_prec, train_rec, train_f1 = compute_metrics(y_true_train, y_pred_train)
test_acc, test_prec, test_rec, test_f1 = compute_metrics(y_true_test, y_pred_test)


metrics_df = pd.DataFrame({
    "Split": ["Train", "Test"],
    "Accuracy": [train_acc, test_acc],
    "Precision": [train_prec, test_prec],
    "Recall": [train_rec, test_rec],
    "F1": [train_f1, test_f1]
})

metrics_df.round(4)

Device set to use cuda:0


,Split,Accuracy,Precision,Recall,F1
0,Train,0.9392,0.9095,0.9031,0.9063
1,Test,0.9328,0.9097,0.8864,0.8973


In [19]:
# check how classifier pipeline actually works
print(classifier(train_df["text"].iloc[0]))

[{'label': 'LABEL_0', 'score': 0.9919050931930542}]
